In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta

In [3]:
#API 콜로 1년 전부터 어제까지 주식데이터 가져오기

# API URL
url = "http://data-dbg.krx.co.kr/svc/apis/sto/stk_bydd_trd"

# 인증키 설정
auth_key = "956087495A1D4769A88B0F3411D0890EF02445EB"  # 제공받은 인증키로 대체하세요

# 헤더작성
headers = {
    "AUTH_KEY": auth_key,

}
# 1년 전 날짜와 어제 날짜 계산
start_date = datetime.today() - timedelta(days=365)
end_date = datetime.today() - timedelta(days=1)


# # 1년 1개월 전 날짜와 어제 날짜 계산
# start_date = datetime.today() - timedelta(days=395)
# end_date = datetime.today() - timedelta(days=30)

# 결과를 저장할 DataFrame
all_data = pd.DataFrame()

# 날짜 반복하여 API 호출
current_date = start_date
while current_date <= end_date:
    bas_dd = current_date.strftime('%Y%m%d')  # YYYYMMDD 형식 변환
    
    # 요청 파라미터
    params = {
        "basDd": bas_dd  # 조회 기준일
    }
    
    # API 요청
    response = requests.get(url, headers=headers, params=params)
    
    # 응답 확인 및 데이터 처리
    if response.status_code == 200:
        data = response.json()
        if 'OutBlock_1' in data and data['OutBlock_1']:  # 데이터가 존재할 경우
            temp_df = pd.DataFrame(data['OutBlock_1'])
            all_data = pd.concat([all_data, temp_df], ignore_index=True)
            print(f"데이터 가져오기 성공: {bas_dd}")
        else:
            print(f"데이터 없음: {bas_dd}")
    else:
        print(f"Error {response.status_code} on {bas_dd}: {response.text}")
    
    # 다음 날짜로 이동
    current_date += timedelta(days=1)

데이터 가져오기 성공: 20231123
데이터 가져오기 성공: 20231124
데이터 없음: 20231125
데이터 없음: 20231126
데이터 가져오기 성공: 20231127
데이터 가져오기 성공: 20231128
데이터 가져오기 성공: 20231129
데이터 가져오기 성공: 20231130
데이터 가져오기 성공: 20231201
데이터 없음: 20231202
데이터 없음: 20231203
데이터 가져오기 성공: 20231204
데이터 가져오기 성공: 20231205
데이터 가져오기 성공: 20231206
데이터 가져오기 성공: 20231207
데이터 가져오기 성공: 20231208
데이터 없음: 20231209
데이터 없음: 20231210
데이터 가져오기 성공: 20231211
데이터 가져오기 성공: 20231212
데이터 가져오기 성공: 20231213
데이터 가져오기 성공: 20231214
데이터 가져오기 성공: 20231215
데이터 없음: 20231216
데이터 없음: 20231217
데이터 가져오기 성공: 20231218
데이터 가져오기 성공: 20231219
데이터 가져오기 성공: 20231220
데이터 가져오기 성공: 20231221
데이터 가져오기 성공: 20231222
데이터 없음: 20231223
데이터 없음: 20231224
데이터 없음: 20231225
데이터 가져오기 성공: 20231226
데이터 가져오기 성공: 20231227
데이터 가져오기 성공: 20231228
데이터 없음: 20231229
데이터 없음: 20231230
데이터 없음: 20231231
데이터 없음: 20240101
데이터 가져오기 성공: 20240102
데이터 가져오기 성공: 20240103
데이터 가져오기 성공: 20240104
데이터 가져오기 성공: 20240105
데이터 없음: 20240106
데이터 없음: 20240107
데이터 가져오기 성공: 20240108
데이터 가져오기 성공: 20240109
데이터 가져오기 성공: 20240110
데이터 가져오

In [55]:
# 최종 데이터 확인 및 저장
#krx_data.csv로 저장.

if not all_data.empty:
    print("데이터 조회 완료")
    print(all_data.head())
    all_data.to_csv("krx_data.csv", index=False, encoding="utf-8-sig")
else:
    print("조회된 데이터가 없습니다.")

데이터 조회 완료
     BAS_DD  ISU_CD   ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC  \
0  20231123  095570   AJ네트웍스  KOSPI          -       4160           -40   
1  20231123  006840    AK홀딩스  KOSPI          -      17990           110   
2  20231123  027410      BGF  KOSPI          -       3530           -25   
3  20231123  282330   BGF리테일  KOSPI          -     131100         -1200   
4  20231123  138930  BNK금융지주  KOSPI          -       7110            40   

  FLUC_RT TDD_OPNPRC TDD_HGPRC TDD_LWPRC ACC_TRDVOL  ACC_TRDVAL  \
0   -0.95       4200      4200      4145      46902   195536505   
1    0.62      17870     18010     17780       3677    65933820   
2   -0.70       3580      3580      3525      57335   203175895   
3   -0.91     132600    133300    131100      60239  7950025700   
4    0.57       7070      7130      7060     538869  3820886230   

          MKTCAP  LIST_SHRS  
0   188251477440   45252759  
1   238323622390   13247561  
2   337880272230   95716791  
3  2265920076600

In [53]:
# 연간 수익률 계산 코드

# 마지막 날짜 확인
latest_date = all_data['BAS_DD'].max()

# 연간 데이터 필터링 (마지막 날짜 기준 같은 연도 데이터)
latest_year = str(latest_date)[:4]  # 연도만 추출
yearly_data = all_data[all_data['BAS_DD'].astype(str).str.startswith(latest_year)]

# FLUC_RT를 숫자로 변환
yearly_data['FLUC_RT'] = pd.to_numeric(yearly_data['FLUC_RT'], errors='coerce')

# 종목별 연간 주가 변동폭 계산
yearly_fluc_rt = yearly_data.groupby('ISU_NM')['FLUC_RT'].sum().reset_index()
yearly_fluc_rt.rename(columns={'FLUC_RT': 'Yearly_FLUC_RT'}, inplace=True)

yearly_fluc_rt

/var/folders/_t/w69h1jqn26q43r4mwvlg_c1m0000gn/T/ipykernel_60420/4172981051.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yearly_data['FLUC_RT'] = pd.to_numeric(yearly_data['FLUC_RT'], errors='coerce')


,ISU_NM,Yearly_FLUC_RT
0,AJ네트웍스,-4.95
1,AK홀딩스,-42.77
2,BGF,-5.98
3,BGF리테일,-14.49
4,BNK금융지주,35.95
...,...,...
982,휴비스,-43.60
983,휴스틸,-14.16
984,흥국화재,27.99
985,흥국화재우,-4.10


In [49]:
# 시총기준 상위 200개 기업 정렬

# MKTCAP 칼럼을 숫자로 변환
all_data['MKTCAP'] = pd.to_numeric(all_data['MKTCAP'], errors='coerce')

# 변환 후 결측값이 있는지 확인
print(all_data['MKTCAP'].isnull().sum())

# 가장 최근 날짜 찾기
latest_date = all_data['BAS_DD'].max()

# 최신 날짜 데이터 필터링
latest_data = all_data[all_data['BAS_DD'] == latest_date]

# MKTCAP 기준 상위 200개 기업 정렬
top_200_companies = latest_data.sort_values(by='MKTCAP', ascending=False).head(200)

# 필요한 칼럼 정리 (ISU_NM, MKTCAP)
result = top_200_companies[['ISU_NM','TDD_CLSPRC', 'MKTCAP']]

# 결과 확인
result

0


,ISU_NM,TDD_CLSPRC,MKTCAP
231509,삼성전자,56400,336695735820000
231230,SK하이닉스,168800,122886799212000
231160,LG에너지솔루션,400500,93717000000000
231504,삼성바이오로직스,932000,66334168000000
231979,현대차,215500,45129189160500
...,...,...,...
231093,ESR켄달스퀘어리츠,4805,1023892645000
231711,이수스페셜티케미컬,33150,1001404482000
231971,현대백화점,42750,1000454352750
231413,두산퓨얼셀,15020,983715764520


,ISU_NM,TDD_CLSPRC,MKTCAP
231509,삼성전자,56400,336695735820000
231230,SK하이닉스,168800,122886799212000
231160,LG에너지솔루션,400500,93717000000000
231504,삼성바이오로직스,932000,66334168000000
231979,현대차,215500,45129189160500
...,...,...,...
231093,ESR켄달스퀘어리츠,4805,1023892645000
231711,이수스페셜티케미컬,33150,1001404482000
231971,현대백화점,42750,1000454352750
231413,두산퓨얼셀,15020,983715764520


In [47]:
# 상위 200개 기업에 연간 수익률 붙이기.

# result에 포함된 종목만 필터링
filtered_fluc_rt = yearly_fluc_rt[yearly_fluc_rt['ISU_NM'].isin(result['ISU_NM'])]

# result 데이터프레임과 연간 변동폭 데이터 병합
result_with_fluc = result.merge(filtered_fluc_rt, on='ISU_NM', how='left')

# 결과 확인
print(result_with_fluc)

         ISU_NM TDD_CLSPRC           MKTCAP  Yearly_FLUC_RT
0          삼성전자      56400  336695735820000          -28.50
1        SK하이닉스     168800  122886799212000           28.61
2      LG에너지솔루션     400500   93717000000000            0.54
3      삼성바이오로직스     932000   66334168000000           24.31
4           현대차     215500   45129189160500           12.64
..          ...        ...              ...             ...
195  ESR켄달스퀘어리츠       4805    1023892645000           29.91
196   이수스페셜티케미컬      33150    1001404482000           41.36
197       현대백화점      42750    1000454352750          -15.73
198       두산퓨얼셀      15020     983715764520          -33.67
199      SK네트웍스       4425     979154716350          -16.89

[200 rows x 4 columns]
